In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
def MeanError(estimates, actual):
    errors = [estimate-actual for estimate in estimates]
    return np.mean(errors)

def SimulateSample(alpha, lamb, size):
    """
    Generate a random sample of inputted size from the Gamma Distribution.
    Use method of moments to estimate alpha / lambda where:
    - Alpha = mean^2 / variance
    - Lambda = Alpha / mean
    """
    data = np.random.gamma(alpha, lamb, size)
    mean = data.mean()
    var = data.var()
    alpha_test = mean**2 / var
    lamb_test = mean / alpha_test
    return alpha_test, lamb_test

def RunTrials(alpha, lamb, size, n):
    """
    Run SimulateSample n times, and calculate the mean error over 
    those n trials for alpha and lambda to see if the method of moments is biased.
    """
    alpha_data = []
    lambda_data = []
    for _ in range(n):
        alpha_test, lamb_test = SimulateSample(alpha, lamb, size)
        alpha_data.append(alpha_test)
        lambda_data.append(lamb_test)
    alpha_error = MeanError(alpha_data, alpha)
    lambda_error = MeanError(lambda_data, lamb)
    return alpha_error, lambda_error

def CheckBias(alpha, lamb, sizes, n):
    """
    Conduct n trials for each size in sizes (a list of ints)
    and calculate mean error so we can see if the bias converges to 0
    as sample size increases.
    """
    alpha_errors = []
    lambda_errors = []
    for size in sizes:
        alpha_error, lambda_error = RunTrials(alpha, lamb, size, n)
        alpha_errors.append(alpha_error)
        lambda_errors.append(lambda_error)
    df = pd.DataFrame({
        "Mean Error (Alpha)": alpha_errors,
        "Mean Error (Lambda)": lambda_errors
    }, index = sizes)

    return df



In [3]:
## Mean = alpha / lambda
## Variance = alpha / lambda^2

alpha = 1 # shape
lamb = 2 # scale
n = 1000
sizes = [_ * 10 for _ in range(1, 10)] + [_ * 100 for _ in range(1, 10)]
df = CheckBias(alpha, lamb, sizes, 100)

fig = px.line(df, x = df.index, y = ["Mean Error (Alpha)", "Mean Error (Lambda)"])
fig.update_layout(
    title = f"Mean Error of Alpha / Lambda for Gamma Distribution vs. Sample Size",
    xaxis_title = "Sample Size",
    yaxis_title = "Mean Error"
)